In [2]:
!pip install category_encoders

In [3]:
import pandas as pd 
import numpy as np
import csv
import matplotlib.pyplot as plt 
import matplotlib 
import warnings
warnings.simplefilter("ignore")

In [4]:
df= pd.read_csv('matches.csv')
df

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,poss,attendance,captain,formation,referee,match report,notes,gca,season,team
0,1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,...,75.0,62443.0,İlkay Gündoğan,4-3-3,Michael Oliver,Match Report,NaN,4.0,2024,Manchester City
1,2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,...,67.0,53453.0,İlkay Gündoğan,4-2-3-1,David Coote,Match Report,NaN,6.0,2024,Manchester City
2,3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,...,69.0,52258.0,İlkay Gündoğan,4-3-3,Jarred Gillett,Match Report,NaN,6.0,2024,Manchester City
3,4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,...,74.0,53112.0,Kevin De Bruyne,4-2-3-1,Darren England,Match Report,NaN,7.0,2024,Manchester City
4,5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,...,74.0,53409.0,İlkay Gündoğan,4-2-3-1,Paul Tierney,Match Report,NaN,9.0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0,2,...,55.0,40290.0,Grant Hanley,4-2-3-1,John Brooks,Match Report,NaN,0.0,2023,Norwich City
1516,39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0,4,...,37.0,26428.0,Grant Hanley,4-2-3-1,Robert Jones,Match Report,NaN,0.0,2023,Norwich City
1517,40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0,3,...,35.0,38092.0,Grant Hanley,4-1-4-1,Simon Hooper,Match Report,NaN,0.0,2023,Norwich City
1518,41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1,1,...,36.0,31219.0,Grant Hanley,3-4-3,Tony Harrington,Match Report,NaN,1.0,2023,Norwich City


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1520 non-null   int64  
 1   date          1520 non-null   object 
 2   time          1520 non-null   object 
 3   comp          1520 non-null   object 
 4   round         1520 non-null   object 
 5   day           1520 non-null   object 
 6   venue         1520 non-null   object 
 7   result        1520 non-null   object 
 8   gf            1520 non-null   int64  
 9   ga            1520 non-null   int64  
 10  opponent      1520 non-null   object 
 11  xg            1520 non-null   float64
 12  xga           1520 non-null   float64
 13  poss          1520 non-null   float64
 14  attendance    1518 non-null   float64
 15  captain       1520 non-null   object 
 16  formation     1520 non-null   object 
 17  referee       1520 non-null   object 
 18  match report  1520 non-null 

In [6]:
df['referee'].value_counts()

Anthony Taylor       116
Paul Tierney         114
Michael Oliver       112
Craig Pawson          94
Stuart Attwell        90
Simon Hooper          88
David Coote           82
Andy Madley           80
Robert Jones          76
Darren England        66
Peter Bankes          66
Andre Marriner        64
Chris Kavanagh        62
Martin Atkinson       52
Mike Dean             52
Jarred Gillett        52
Jonathan Moss         50
John Brooks           46
Graham Scott          36
Michael Salisbury     36
Kevin Friend          36
Tony Harrington       18
Thomas Bramall        16
Darren Bond            8
Tim Robinson           4
Robert Madley          2
Joshua Smith           2
Name: referee, dtype: int64

In [7]:
df.drop(['Unnamed: 0','attendance','captain','xga','xg','notes','match report','season','comp'], axis=1, inplace=True)

In [8]:
df['time'].unique()

array(['16:30', '15:00', '19:30', '17:30', '12:30', '14:00', '20:00',
       '16:00', '12:00', '19:45', '19:00', '20:15', '16:15', '13:30',
       '14:15', '18:00'], dtype=object)

In [9]:
df["date"] = pd.to_datetime(df["date"])

In [10]:
df["venue_code"] = df["venue"].astype("category").cat.codes
df["opp_code"] =df["opponent"].astype("category").cat.codes
df["hour"] = df["time"].str.replace(":.+", "", regex=True).astype("int")
df["day_code"] = df["date"].dt.dayofweek
df['ref_code'] = df['referee'].astype('category').cat.codes
df['formation_codes']=df['formation'].astype('category').cat.codes

In [12]:
from category_encoders.ordinal import OrdinalEncoder
encoder= OrdinalEncoder(cols=['result'],return_df=True,
 mapping=[{'col':'result',
'mapping':{'W':3,'D':1,'L':0}}])
df['results'] = encoder.fit_transform(df['result'])

In [13]:
df

,date,time,round,day,venue,result,gf,ga,opponent,poss,...,referee,gca,team,venue_code,opp_code,hour,day_code,ref_code,formation_codes,results
0,2022-08-07,16:30,Matchweek 1,Sun,Away,W,2,0,West Ham,75.0,...,Michael Oliver,4.0,Manchester City,0,21,16,6,15,14,3
1,2022-08-13,15:00,Matchweek 2,Sat,Home,W,4,0,Bournemouth,67.0,...,David Coote,6.0,Manchester City,1,2,15,5,7,11,3
2,2022-08-21,16:30,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,69.0,...,Jarred Gillett,6.0,Manchester City,0,15,16,6,9,14,1
3,2022-08-27,15:00,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,74.0,...,Darren England,7.0,Manchester City,1,7,15,5,6,11,3
4,2022-08-31,19:30,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,74.0,...,Paul Tierney,9.0,Manchester City,1,17,19,2,18,11,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,2022-04-30,15:00,Matchweek 35,Sat,Away,L,0,2,Aston Villa,55.0,...,John Brooks,0.0,Norwich City,0,1,15,5,10,11,0
1516,2022-05-08,14:00,Matchweek 36,Sun,Home,L,0,4,West Ham,37.0,...,Robert Jones,0.0,Norwich City,1,21,14,6,20,11,0
1517,2022-05-11,19:45,Matchweek 21,Wed,Away,L,0,3,Leicester City,35.0,...,Simon Hooper,0.0,Norwich City,0,11,19,2,22,9,0
1518,2022-05-15,14:00,Matchweek 37,Sun,Away,D,1,1,Wolves,36.0,...,Tony Harrington,1.0,Norwich City,0,22,14,6,26,4,1


In [14]:
grouped_matches=df.groupby('team')

In [15]:
group = grouped_matches.get_group("Manchester City").sort_values("date")
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group
cols = ["gf", "ga", "gca","poss"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,date,time,round,day,venue,result,gf,ga,opponent,poss,...,opp_code,hour,day_code,ref_code,formation_codes,results,gf_rolling,ga_rolling,gca_rolling,poss_rolling
763,2021-09-11,15:00,Matchweek 4,Sat,Away,W,1,0,Leicester City,61.0,...,11,15,5,18,14,3,3.333333,0.333333,5.666667,70.666667
764,2021-09-18,15:00,Matchweek 5,Sat,Home,D,0,0,Southampton,64.0,...,18,15,5,11,14,1,3.666667,0.000000,6.000000,69.333333
765,2021-09-25,12:30,Matchweek 6,Sat,Away,W,1,0,Chelsea,59.0,...,6,12,5,15,14,3,2.000000,0.000000,3.666667,68.333333
766,2021-10-03,16:30,Matchweek 7,Sun,Away,D,2,2,Liverpool,51.0,...,12,16,6,18,14,1,0.666667,0.000000,0.666667,61.333333
767,2021-10-16,15:00,Matchweek 8,Sat,Home,W,2,0,Burnley,70.0,...,5,15,5,14,14,3,1.000000,0.666667,1.666667,58.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023-05-06,15:00,Matchweek 35,Sat,Home,W,2,1,Leeds United,80.0,...,10,15,5,1,1,3,3.000000,0.666667,5.666667,61.000000
34,2023-05-14,14:00,Matchweek 36,Sun,Away,W,3,0,Everton,63.0,...,8,14,6,2,11,3,2.333333,0.666667,4.666667,70.333333
35,2023-05-21,16:00,Matchweek 37,Sun,Home,W,1,0,Chelsea,64.0,...,6,16,6,15,5,3,2.666667,0.333333,5.000000,70.666667
36,2023-05-24,20:00,Matchweek 32,Wed,Away,D,1,1,Brighton,60.0,...,4,20,2,22,14,1,2.000000,0.333333,3.666667,69.000000


In [16]:
df_rolling = df.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [17]:
df_rolling = df_rolling.droplevel('team')

In [18]:
df_rolling.reset_index(drop='index',inplace=True)

In [19]:
df_rolling.columns

Index(['date', 'time', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'poss', 'formation', 'referee', 'gca', 'team', 'venue_code',
       'opp_code', 'hour', 'day_code', 'ref_code', 'formation_codes',
       'results', 'gf_rolling', 'ga_rolling', 'gca_rolling', 'poss_rolling'],
      dtype='object')

In [20]:
df_rolling.drop(['time', 'round', 'day', 'venue', 'gf', 'ga',
        'poss', 'formation', 'referee', 'gca'], axis=1, inplace= True)

In [21]:
df_rolling

,date,result,opponent,team,venue_code,opp_code,hour,day_code,ref_code,formation_codes,results,gf_rolling,ga_rolling,gca_rolling,poss_rolling
0,2021-09-11,W,Norwich City,Arsenal,1,16,15,5,15,11,3,0.000000,3.000000,0.000000,39.666667
1,2021-09-18,W,Burnley,Arsenal,0,5,15,5,2,9,3,0.333333,2.333333,0.333333,35.333333
2,2021-09-26,W,Tottenham,Arsenal,1,19,16,6,4,11,3,0.666667,1.666667,1.000000,41.666667
3,2021-10-02,D,Brighton,Arsenal,0,4,17,5,11,11,1,1.666667,0.333333,3.000000,50.333333
4,2021-10-18,D,Crystal Palace,Arsenal,1,7,20,0,17,9,1,1.333333,0.333333,2.666667,47.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,2023-04-29,L,Brighton,Wolverhampton Wanderers,0,4,15,5,7,16,0,1.666667,0.666667,2.000000,49.666667
1447,2023-05-06,W,Aston Villa,Wolverhampton Wanderers,1,1,15,5,23,16,3,1.000000,2.666667,0.666667,47.333333
1448,2023-05-13,L,Manchester Utd,Wolverhampton Wanderers,0,14,15,5,10,16,0,1.000000,2.000000,1.000000,40.666667
1449,2023-05-20,D,Everton,Wolverhampton Wanderers,1,8,15,5,7,16,1,0.333333,2.666667,0.666667,44.333333


In [22]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

In [23]:
predictors=['venue_code','opp_code','ref_code','formation_codes'] + new_cols

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [27]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-06-01']
    test = data[data["date"] > '2022-06-01']
    clf.fit(train[predictors], train["results"])
    preds = clf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["results"], predicted=preds), index=test.index)
    error = precision_score(test["results"], preds, average=None)
    return combined, error
combined, error = make_predictions(df_rolling, predictors)

In [31]:
combined = combined.merge(df_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [32]:
combined.sample(10)

,actual,predicted,date,team,opponent,result
53,3,1,2023-01-22,Arsenal,Manchester Utd,W
147,1,0,2022-08-31,Bournemouth,Wolves,D
1450,0,0,2023-05-28,Wolverhampton Wanderers,Arsenal,L
890,0,3,2023-02-05,Manchester City,Tottenham,L
1252,3,0,2023-01-23,Tottenham Hotspur,Fulham,W
119,3,3,2022-10-23,Aston Villa,Brentford,W
144,1,3,2023-05-20,Aston Villa,Liverpool,D
725,0,0,2022-08-13,Leicester City,Arsenal,L
592,0,0,2022-11-13,Fulham,Manchester Utd,L
419,0,3,2023-02-18,Chelsea,Southampton,L


In [34]:
import pickle

with open('footypredict.pkl', 'wb') as f:
    pickle.dump(clf, f)